In [ ]:
# Code derived from: https://github.com/bioinf-jku/TTUR

import os
import glob
#os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
import fid
from matplotlib.pyplot import imread
#import tensorflow as tf
import tensorflow.compat.v1 as tf # The is score was made in tensorflow v1
tf.disable_v2_behavior()          # These two lines solves the problem

########
# PATHS
########
data_path = '../input/afhq64x64/afhq64x64/' # set path to training set images
output_path = 'fid_stats.npz' # path for where to store the statistics

inception_path = None
print("check for inception model..", end=" ", flush=True)
inception_path = fid.check_or_download_inception(inception_path) # download inception if necessary
print("ok")

# loads all images into memory (this might require a lot of RAM!)
print("load images..", end=" " , flush=True)
image_list = glob.glob(os.path.join(data_path, '*.png'))
images = np.array([imread(str(fn)).astype(np.float32) for fn in image_list])
print("%d images found and loaded" % len(images))

print("create inception graph..", end=" ", flush=True)
fid.create_inception_graph(inception_path)  # load the graph into the current TF graph
print("ok")

print("calculte FID stats..", end=" ", flush=True)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    mu, sigma = fid.calculate_activation_statistics(images, sess, batch_size=100)
    np.savez_compressed(output_path, mu=mu, sigma=sigma)
print("finished")